In [1]:
import time
from pyswarm import pso

from upapy.io import load_data
from upapy.reconstruction.ubp2d import FastUBP
from focus_func import *

In [2]:
# 批量重构函数
def batch_reconstruct(
    data: np.ndarray, sos: float = 1.5, image_size: int = 30, half_time: int = 0
) -> np.ndarray:
    if data.ndim == 3:
        fubp = FastUBP()
        fubp.set_pa_frame_element_number(data.shape[1])
        fubp.set_reconstruction_water_sos(sos)
        fubp.set_reconstruction_tissue_sos(sos)
        fubp.set_image_length(image_size)
        fubp.set_image_width(image_size)
        fubp.set_control_half_time_delta(half_time)
        recon_list = []
        for i in range(data.shape[0]):
            recon_slice = fubp.reconstruction(data[i])
            recon_list.append(recon_slice)
        return np.array(recon_list)
    else:
        raise ValueError("Incorrect array dimension.")

In [3]:
data = load_data("../data/mouse_data/D11980_1064 nm_30 %_20db_mouse_000.pah5")
data = data[50:150]
data.shape

(100, 512, 2000)

In [4]:
# 迭代统计数据
iteration_stats = []

# 定义目标函数
def objective_function(sos):
    recon_start = time.time()
    recon = batch_reconstruct(data, sos)
    recon_end = time.time()
    recon_time = recon_end - recon_start

    mip = np.max(-recon, axis=0)
    gradient = calculate_tenenbaum_gradient(mip[np.newaxis,])

    # 记录本次迭代的统计信息
    iteration_stats.append({
        "sos": sos,
        "recon_time": recon_time,
    })

    return -gradient

# 声速下限与上限
lb = [1.45]
ub = [1.55]

# PSO 优化
optim_start = time.time()
xopt, fopt = pso(objective_function, lb, ub, swarmsize=5, maxiter=10)
optim_end = time.time()

# 计算总体统计
total_recon_time = sum(stat["recon_time"] for stat in iteration_stats)
optim_time = optim_end - optim_start

print("最优解:", xopt)
print("目标值:", fopt)
print("总体重构耗时:", round(total_recon_time, 2))
print("总体优化耗时:", round(optim_time, 2))
print("重构耗时占比:", round(total_recon_time / optim_time, 4)*100, "%")

Stopping search: maximum iterations reached --> 10
最优解: [1.5089945]
目标值: [-50545044.]
总体重构耗时: 169.27
总体优化耗时: 178.0
重构耗时占比: 95.09 %


In [5]:
# 定义目标函数
def objective_function(sos):
    recon = batch_reconstruct(data, sos)
    mip = np.max(-recon, axis=0)
    gradient = calculate_tenenbaum_gradient(mip[np.newaxis,])

    return -gradient

# 构建优化器所需的辅助函数
def generate_discrete_values(lb, ub, precision):
    decimal_places = len(str(precision).split('.')[1]) if '.' in str(precision) else 0
    return np.round(np.arange(lb, ub + precision, precision), decimal_places)

# 自定义 PSO 优化器
def custom_pso(objective_function, lb, ub, precision=0.001, swarmsize=10, maxiter=20, w=0.5, c1=1.5, c2=1.5):
    discrete_values = generate_discrete_values(lb, ub, precision)
    history = dict()

    def evaluate(sos):
        if sos in history:
            return history[sos]
        val = objective_function(sos)
        history[sos] = val
        return val

    # 初始化粒子
    # particles = np.random.choice(discrete_values, size=swarmsize)
    # 选择每个分段的中间位置作为粒子初始位置
    segment_size = len(discrete_values) / swarmsize
    particles = np.array([
        discrete_values[int(i * segment_size + segment_size / 2)]
        for i in range(swarmsize)
    ])
    velocities = np.zeros(swarmsize)

    personal_best = particles.copy()
    personal_best_values = np.array([evaluate(p) for p in personal_best])

    global_best_idx = np.argmin(personal_best_values)
    global_best = personal_best[global_best_idx]
    global_best_value = personal_best_values[global_best_idx]
    # print(type(global_best), type(global_best_value))
    for iter in range(maxiter):
        for i in range(swarmsize):
            r1 = np.random.rand()
            r2 = np.random.rand()

            # 计算新速度
            inertia = w * velocities[i]
            cognitive = c1 * r1 * (personal_best[i] - particles[i])
            social = c2 * r2 * (global_best - particles[i])
            velocities[i] = inertia + cognitive + social

            # 更新位置（注意：必须从离散值中选最近的）
            new_position = particles[i] + velocities[i]
            closest = discrete_values[np.argmin(np.abs(discrete_values - new_position))]
            particles[i] = closest

            # 评估
            val = evaluate(particles[i])
            if val < personal_best_values[i]:
                personal_best[i] = particles[i]
                personal_best_values[i] = val

        # 更新全局最优
        global_best_idx = np.argmin(personal_best_values)
        if personal_best_values[global_best_idx] < global_best_value:
            global_best = personal_best[global_best_idx]
            global_best_value = personal_best_values[global_best_idx]

        # print(f"Iteration {iter + 1}, Best sos = {global_best:.4f}, Score = {global_best_value.item():.0f}")

    return global_best, global_best_value, history

In [6]:
best_sos, best_score, cache = custom_pso(objective_function,
                                         lb=1.45, ub=1.55, precision=0.0005,
                                         swarmsize=6, maxiter=20,
                                         w=0.2, c1=0, c2=1.0)

print("最优声速:", best_sos)
print("最优得分:", best_score)
print("History:", len(cache))

sos_list = []
for sos in cache:
    sos_list.append(round(float(sos), 4))
sos_list.sort()
print(sos_list)

最优声速: 1.509
最优得分: [-50545392.]
History: 27
[1.458, 1.466, 1.475, 1.483, 1.492, 1.497, 1.501, 1.503, 1.5035, 1.505, 1.5055, 1.5065, 1.507, 1.5075, 1.508, 1.5085, 1.509, 1.5095, 1.51, 1.5105, 1.511, 1.512, 1.5145, 1.515, 1.519, 1.5255, 1.5425]
